## <i> Creating calendar in Google calendar using event data from Google sheets </i>


<strong> Aim: </strong> To read event creation raw data from a Google Sheet such as Event title, summary, start datetime, end datetime, address, email ID(of the individual for whom the event will be created) and to create Google calendar events for the individuals mentioned in the email field using the said raw data via Google Calendar API.
<p>
    <strong> Google sheet link: </strong> https://docs.google.com/spreadsheets/d/1XYQG_mAisQrNRMiRpOCCHPIJuy-T9RbaEtURvv_O5Os/edit?usp=sharing

## fetch raw data + create event dicts

In [10]:
# imports

#-----Google Api packages
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
#----

#----file manipulation packages
import time
import pandas as pd
import ast
import datetime
#----

#----standard functionality
import math
from datetime import date
import os
from datetime import datetime
import pprint as pp
#----

In [11]:
#----  Google Sheet CRUD functions

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


def clear2gs(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME, json_credential_file):
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """

    # The ID and range of a sample spreadsheet.

    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                json_credential_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    body = {'valueInputOption': "RAW"}

    request = service.spreadsheets().values().clear(
        spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME,
        body={}).execute()


def write2gs(values, SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME,
             json_credential_file):

    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                json_credential_file, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    data = [
        {
            'range': SAMPLE_RANGE_NAME,
            'values': values
        },
    ]
    body = {'valueInputOption': "RAW", 'data': data}
    result = service.spreadsheets().values().batchUpdate(
        spreadsheetId=SAMPLE_SPREADSHEET_ID, body=body).execute()


def read_gs(SAMPLE_SPREADSHEET_ID, SAMPLE_RANGE_NAME):
    import pprint as pp
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])

    #     for item in result, values:
    #         pp.pprint(item)
    return result, values


# Google Sheets update functions end here

In [12]:
#GSheet to pd.DataFrame functions


def Create_Service(client_secret_file, api_name, api_version, *scopes):
    #     print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    #     print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        #         print(API_SERVICE_NAME, 'service created successfully')
        print('RD fetch auth flow completed successfully!')
        return service
    except Exception as e:
        print('RD fetch auth flow completed failed!')
        print(e)
    return None


def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

In [13]:
# fetch & print the DF

CLIENT_SECRET_FILE = 'credentials.json'
API_SERVICE_NAME = 'sheets'
API_VERSION = 'v4'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
gsheetId = '1XYQG_mAisQrNRMiRpOCCHPIJuy-T9RbaEtURvv_O5Os'

s = Create_Service(CLIENT_SECRET_FILE, API_SERVICE_NAME, API_VERSION, SCOPES)
gs = s.spreadsheets()
rows = gs.values().get(spreadsheetId=gsheetId,
                       range='Create Events!A1:O100').execute()
data = rows.get('values')
raw_df = pd.DataFrame(data)
print('Raw DF fetched!')
# print("\n\n", "-" * 58 , "RAW - DF", "-" * 58)
# raw_df

RD fetch auth flow completed successfully!
Raw DF fetched!


In [14]:
# create a df containing only the rows for which events are yet to be created
# logic of Y_colm --> Every time an event is created, a Y is put in the google sheet indicating successful event creation
# in order to avoid re-creating events, I've employ a log that avoids rows which have a "Y" in the  event_created column
Y_colm = pd.Series(raw_df[12])
last_Y = Y_colm.where(Y_colm == 'Y').last_valid_index() + 1

df = raw_df.iloc[last_Y:]
df

############################ add a condition here to only continue if df len > 0

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
5,Devansh D. Sharma,01_devansh@domain.com,Summary_4,Please go to Summary_4 at timings mentioned in...,Address_4,2020-05-19,9:00:00,2020-05-19,18:00:00,2020-05-19T9:00:00.000+02:00,2020-05-19T18:00:00.000+02:00,None,None,None


In [15]:
# convert each row to a dict

tst_dict = df.to_dict
row_dicts = df.to_dict(orient='records')

In [16]:
# put all dicts in one list - event_dict_RD

events_master_lst = []  # master event list

for i in range(len(row_dicts)):
    pp.pprint(row_dicts[i])
    events_master_lst.append(row_dicts[i])
    print('\n')

{0: 'Devansh D. Sharma',
 1: '01_devansh@domain.com',
 2: 'Summary_4',
 3: 'Please go to Summary_4 at timings mentioned in the event.',
 4: 'Address_4',
 5: '2020-05-19',
 6: '9:00:00',
 7: '2020-05-19',
 8: '18:00:00',
 9: '2020-05-19T9:00:00.000+02:00',
 10: '2020-05-19T18:00:00.000+02:00',
 11: None,
 12: None,
 13: None}




In [17]:
# create event dicts for each event using the [event_dict_RD]
# These are the final event dicts sent to Google API to create events

event_dict_RD = []

for event in events_master_lst:
    email = event.get(1)
    title = event.get(2)
    description = event.get(3)
    address = event.get(4)
    ev_start = event.get(9)
    ev_end = event.get(10)

    event_dict_from_rd = {
        'summary': title,
        'location': address,
        'description': description,
        'start': {
            'dateTime': str(ev_start),
            'timeZone': "Europe/Paris",
        },
        'end': {
            'dateTime': str(ev_end),
            'timeZone': "Europe/Paris",
        },
        #       'recurrence': [
        #         'RRULE:FREQ=DAILY;COUNT=1'
        #       ],
        'attendees': [{
            'email': email
        }],
        'reminders': {
            'useDefault':
            False,
            'overrides': [
                {
                    'method': 'email',
                    'minutes': 24 * 60
                },
                {
                    'method': 'popup',
                    'minutes': 10
                },
            ],
        },
    }

    print(event_dict_from_rd, '\n')
    event_dict_RD.append(event_dict_from_rd)

{'summary': 'Summary_4', 'location': 'Address_4', 'description': 'Please go to Summary_4 at timings mentioned in the event.', 'start': {'dateTime': '2020-05-19T9:00:00.000+02:00', 'timeZone': 'Europe/Paris'}, 'end': {'dateTime': '2020-05-19T18:00:00.000+02:00', 'timeZone': 'Europe/Paris'}, 'attendees': [{'email': '01_devansh@domain.com'}], 'reminders': {'useDefault': False, 'overrides': [{'method': 'email', 'minutes': 1440}, {'method': 'popup', 'minutes': 10}]}} 



### Event Creation

In [18]:
# auth for event creation

SCOPES = ['https://www.googleapis.com/auth/calendar']
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token_for_event_creation.pickle'):
    with open('token_for_event_creation.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials_gCal.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token_for_event_creation.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [ ]:
# create the events & store info like event link & event IDs into lists

event_ids_list = []
ev_links = []

for i in range(len(event_dict_RD)):
    service = build('calendar', 'v3', credentials=creds)
    event = service.events().insert(calendarId='primary',
                                    sendUpdates='all',
                                    body=event_dict_RD[i]).execute()
    ev_id = event.get('id')
    event_link = event.get('htmlLink')
    ev_links.append(event_link)
    event_ids_list.append(ev_id)
    print("#", i + 1, "|", "ev_id: ", ev_id, event_dict_RD[i].get('summary'),
          "//", 'Event created: ', event_link, "\n")

## Write feedback to Google Sheet

In [ ]:
# create a DataFrame of feedback

responses_lst = []

df_len = len(df)

for l in range(last_Y, df_len + last_Y):
    responses_lst.append('Y')

df_feedback = pd.DataFrame(data=ev_links)
df_feedback[1] = pd.DataFrame(data=responses_lst)

df_feedback

In [ ]:
# write the DF to the sheet

write2gs(df_feedback.where((pd.notnull(df_feedback)), None).values.tolist(),'1FjQ1b1OQdEkgRk9F6C4x99MAXUqC1DpgU8De5rRUSr4', 'Create Events for Buyers!L' + str(last_Y + 1 ) + ":M" + str(df_feedback.shape[0] + last_Y), creds)

### Reference
* Sheet link - https://docs.google.com/spreadsheets/d/1XYQG_mAisQrNRMiRpOCCHPIJuy-T9RbaEtURvv_O5Os/edit?usp=sharing
* [Recurrence 1]: https://developers.google.com/calendar/concepts/events-calendars
* [Recurrence 2] : https://developers.google.com/calendar/recurringevents
* [Insert] : https://developers.google.com/calendar/v3/reference/events/insert
* [Full event dict structure] : https://developers.google.com/calendar/v3/reference/events